<a href="https://colab.research.google.com/github/lucifernandes/dadosSociaisSVM/blob/master/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pré processamento

##Dados

In [ ]:
import pandas as pd
base = pd.read_csv('/content/drive/My Drive/Bases/base_v01.csv', sep=',', decimal=',')
base = base.dropna(how="any") 
base = base.reset_index(drop=True)
base.head()

In [ ]:
base['renda'] = (base['renda']).astype(int)

dadosSocio = base.loc[:, ['idade', 'sM', 'sF', 'corBr',	'corPr','corPa','corAm','corIn']]
cbc = base.loc[:, ['carro','empreg','maqlav','dvd','banhe','gelad','freezer','comput',
                          'lavlou','microon','motocic','secadora','agua','rua','escola','renda']]
bloco1 = base.loc[:,['p41aa','p41ab','p41ad','p41ae','p41af','p41ag','p41ah','p41ai',
                     'p41aj','p41ak','p41al','p41am','p41an','p41ao','p41ap']]
bloco2 = base.loc[:,['p42a','p42b','p42d','p42e','p42f','p42g','p42h','p42i','p42j','p42k',
                     'p42l','p42m','p42n','p42o','p42p']]
bloco3 = base.loc[:,['p43a','p43b','p43d','p43f','p43g','p43h','p43i','p43j','p43k','p43l',
                     'p43m','p43n','p43o','p43p']]
bloco4 = base.loc[:, ['INV1p45a','INV1p45b','INV1p45c','INV1p45d','INV1p45e','INV1p45f',
                      'INV1p45g','INV1p45h','INV1p45i','INV1p45j','INV1p45k','INV1p45l',
                      'INV1p45m','INV1p45n','INV1p45o','INV1p45p','INV1p45q']].astype(int) 

def binarizar(entrada):
  classe = pd.DataFrame({})
  classe['classe'] = entrada.mean(axis=1)
  print(classe)
  mediana = classe['classe'].median()
  #print('Mediana: ', mediana)
  #print(classe)
  for i in range(len(classe)):
    if classe.iloc[i,0] < mediana:
      classe.iloc[i,0] = 0
    else:
      classe.iloc[i,0] = 1
  classe['classe'] = (classe['classe']).astype(int)
  return classe

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
def normalizar(entrada):
  scaler = MinMaxScaler()
  previsores = scaler.fit_transform(entrada)
  return previsores
def padronizar(entrada):
  scaler = StandardScaler() #Padronização
  previsores = scaler.fit_transform(entrada)
  return previsores

bloco4.head()
#auto['Indbloco1'] = (auto['Indbloco1']).astype(int)
#base['IndAuto'] = A
#base['IndAuto'] = (base['IndAuto']).astype(int)

#Índice Autoritarismo

In [ ]:
indAuto = pd.DataFrame({})
indAuto = pd.concat([dadosSocio,cbc,bloco1,bloco2,bloco3], axis=1)

cAuto = binarizar(bloco4)
cAuto = cAuto.values
indAuto = indAuto.values

#indAuto = normalizar(indAuto)
indAuto = padronizar(indAuto)

        classe
0     5.000000
1     4.529412
2     5.705882
3     5.705882
4     5.529412
...        ...
1752  1.058824
1753  1.529412
1754  1.058824
1755  2.470588
1756  1.529412

[1757 rows x 1 columns]


#Índice Medo do Crime (bloco 1)

In [ ]:
indb1 = pd.DataFrame({})
indb1 = pd.concat([dadosSocio,cbc,bloco2,bloco3,bloco4], axis=1)

cb1 = binarizar(bloco1)

#indb1 = normalizar(indb1)
#indb1 = padronizar(indb1)


#Índice Chances de ocorrência de vitimização por crime (bloco 2)

In [ ]:
indb2 = pd.DataFrame({})
indb2 = pd.concat([dadosSocio,cbc,bloco1,bloco3,bloco4], axis=1)

cb2 = binarizar(bloco2)

#indb2 = normalizar(indb2)
indb2 = padronizar(indb2)


        classe
0     8.066667
1     2.866667
2     5.333333
3     1.933333
4     2.266667
...        ...
1752  7.400000
1753  6.600000
1754  6.533333
1755  2.200000
1756  4.333333

[1757 rows x 1 columns]


#Índice Vitimização do crime (bloco 3)

In [ ]:
indb3 = pd.DataFrame({})
indb3 = pd.concat([dadosSocio,cbc,bloco1,bloco2,bloco4], axis=1)

cb3 = binarizar(bloco3)

#indb3 = normalizar(indb3)
#indb3 = padronizar(indb3)


        classe
0     0.000000
1     0.071429
2     0.000000
3     0.000000
4     0.000000
...        ...
1752  0.142857
1753  0.000000
1754  0.071429
1755  0.142857
1756  0.000000

[1757 rows x 1 columns]


#Índice Critério Brasil de Classes

In [ ]:
indcbc = pd.DataFrame({})
indcbc = pd.concat([dadosSocio,bloco1,bloco2,bloco3,bloco4], axis=1)

ccbc = binarizar(cbc)

#indcbc = normalizar(indcbc)
indcbc = padronizar(indcbc)

##Tentativa com threshold=3.5

In [ ]:
c0 = pd.DataFrame({}, columns=['cbc','bloco1','bloco2','bloco3','idade','sM','sF','corBr','corPr','corPa','corAm','corIn','classe'])
c1 = pd.DataFrame({}, columns=['cbc','bloco1','bloco2','bloco3','idade','sM','sF','corBr','corPr','corPa','corAm','corIn','classe'])

for i in range(len(prev)):
  if prev.loc[i,'classe'] == 0:
    c0.loc[i] = prev.iloc[i,:]
  else:
    c1.loc[i] = prev.iloc[i,:]
  
c0 = c0.reset_index(drop=True)
c1 = c1.reset_index(drop=True)  

from random import sample
import numpy as np
from array import array 
r0 = np.arange(1581)       
r = sample(list(r0),176)



c1n = pd.DataFrame({}, columns=['cbc','bloco1','bloco2','bloco3','idade','sM','sF','corBr','corPr','corPa','corAm','corIn','classe'])
for i in r:
  c1n.loc[i] = c1.iloc[i,:]
c1n = c1n.reset_index(drop=True)  

data = pd.concat([c0, c1n])     #Concatenação dos sinais da Classe 1 com os sinais da Classe 2
data.reset_index(drop=True,inplace=True) #Resetando os índices


clas = data.loc[:,'classe']
data = data.drop(columns=['classe'])

clas.head(352)

#Classificador

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

resultados20 = []
for i in range(20):
  
  previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(indb3, cb3, test_size=0.1, random_state=i)

  #‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
  classificador = SVC(kernel='rbf', random_state = i, C=20)

  classificador.fit(previsores_treinamento, classe_treinamento)
  previsoes = classificador.predict(previsores_teste)

  precisao = accuracy_score(classe_teste, previsoes)
  matriz = confusion_matrix(classe_teste, previsoes)

  resultados20.append(precisao)

for i in range(20):
  print(str(resultados20[i]).replace('.', ','))

In [ ]:
cAuto['classe'].value_counts()

1    882
0    875
Name: classe, dtype: int64